- [Mathematics for Deep Learning](http://www.d2l.ai/chapter_appendix-mathematics-for-deep-learning/index.html)

- 3-d: RGB 图片（宽 * 高 * 通道）
- 4-d: 一个 RGB 图片批量（批量大小 * 宽 * 高 * 通道）
- 5-d: 一个视频批量（批量大小 * 时间 * 宽 * 高 * 通道）

- 创建数组需要：

  - 形状
  - 每个元素的数据类型
  - 每个元素的值

- 访问元素子区域: `[::3, ::2]`
  - 这是一个 python 的切片操作，表示在**第一个**维度上，以**步长**3 进行切片，即取索引为 0、3、6、9 等的列
  - 在**第二个**维度上，以**步长**为 2 进行切片，即取索引为 0、2、4、6 等的列


In [1]:
import torch

In [2]:
x = torch.arange(12)

In [3]:
x.shape     # 查看形状

torch.Size([12])

In [4]:
x.numel()   # 元素的总数

12

In [5]:
x = x.reshape(3, 4)     # 改变一个张量的形状而不改变元素数量和元素值

In [7]:
torch.zeros((2, 3, 4))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [8]:
torch.ones((2, 3, 4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [9]:
torch.tensor([[2, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])

tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [4, 3, 2, 1]])

In [10]:
x = torch.tensor([1.0, 2, 4, 8])    # 创建为浮点的tensor
y = torch.tensor([2, 2, 2, 2])      # 创建为整型的tensor
x + y, x - y, x / y, x**y

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))

In [11]:
torch.exp(x)

tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03])

In [14]:
X = torch.arange(12, dtype=torch.float32).reshape((3, 4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)      # 结果显示两个

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

In [15]:
X == Y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

In [17]:
X.sum()

tensor(66.)

- 即使形状不同，我们仍然可以通过调用广播机制（broadcasting mechanism）来执行按元素操作

In [18]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [19]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

In [20]:
X[-1], X[1:3]   # 最后一行，1-2行

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]))

In [26]:
X[0:2, :] = 12      # 为多个元素赋值相同的值
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])

- 运行一些操作可能会导致为新结果分配内存

In [27]:
before = id(Y)
Y = Y + X
id(Y) == before

False

- 执行原地操作

In [28]:
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))

id(Z): 140707950984784
id(Z): 140707950984784


- 如果在后续计算中没有重复使用`X`，我们也可以使用`X[:] = X + Y` 或 `X += Y` 来减少操作的内存开销

In [29]:
before = id(X)
X += Y
id(X) == before

True

- 转换为NumPy张量

In [30]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

- 将大小为1的张量转换为Python标量

In [31]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)

-------------------------------------------

### 数据预处理
- 创建一个人工数据集，并存储在csv（逗号分隔值）文件

In [1]:
import os

os.makedirs(os.path.join('data'), exist_ok=True)
data_file = os.path.join('data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')   # 列名
    f.write('NA,Pave,127500\n')     # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')